In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math

# Transformer

Let's build the transformer model from bottom up. We will start with the basic building blocks and then put them together to build the transformer model.


In [3]:
# Layer Normalization
class LayerNormalization(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))

    def forward(self, x):
        # X: (batch_size, seq_len, features)
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [4]:
# Embedding Layer
class InputEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding_size = embedding_size

    def forward(self, x):
        # x: (batch_size, seq_len)
        # Scaling the embedding by sqrt(embedding_size) as per the paper
        return self.embedding(x) * math.sqrt(self.embedding_size)

In [5]:
# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()

        # pe: (max_len, d_model)
        pe = torch.zeros(max_len, d_model)

        # position: (max_len, 1)
        position = torch.arange(0, max_len).unsqueeze(1).float()

        # div_term: (d_model // 2)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)
        )

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # pe: (1, max_len, d_model)
        pe = pe.unsqueeze(0)

        self.register_buffer("pe", pe)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, : x.size(1)]
        return self.dropout(x)

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.n_heads = n_heads

        assert d_model % n_heads == 0, "d_model should be divisible by n_heads"
        self.head_dim = d_model // n_heads

        self.qkv_ln = nn.Linear(d_model, 3 * d_model)
        self.out = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):

        q, k, v = map(
            lambda t: t.view(
                t.size(0), t.size(1), self.n_heads, self.head_dim
            ).transpose(1, 2),
            self.qkv_ln(x).chunk(3, dim=-1),
        )

        # Calculate the attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # Apply mask
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Apply softmax
        scores = F.softmax(scores, dim=-1)

        # Apply dropout
        scores = self.dropout(scores)

        # Get attention
        attention = torch.matmul(scores, v)

        # Concat
        attention = (
            attention.transpose(1, 2)
            .contiguous()
            .view(x.size(0), x.size(1), self.d_model)
        )

        return self.out(attention)

In [7]:
# Residual Layer and Layer Normalization
class ResidualLayer(nn.Module):
    def __init__(self, features, dropout=0.1):
        super().__init__()

        self.norm = LayerNormalization(features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return self.dropout(self.norm(x + sublayer(x)))  # Post Layer Normalization

In [8]:
# Feed Forward Layer
class FFN(nn.Module):
    def __init__(
        self,
        d_model,
        d_ff,
        dropout=0.1,
    ):
        super().__init__()

        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

In [9]:
# Encoder Block
class EncoderBlock(nn.Module):
    def __init__(
        self,
        features,
        n_heads,
        d_ff,
        dropout=0.1,
    ):
        super().__init__()

        self.attention = MultiHeadAttention(features, n_heads, dropout)
        self.ffn = FFN(features, d_ff, dropout)
        self.residual = nn.ModuleList(
            [ResidualLayer(features, dropout) for _ in range(2)]
        )

    def forward(self, x, src_mask):
        x = self.residual[0](x, lambda x: self.attention(x, src_mask))
        x = self.residual[1](x, self.ffn)

        return x

In [10]:
# Encoder
class Encoder(nn.Module):
    def __init__(
        self,
        features,
        n_heads,
        d_ff,
        n_layers=8,
        dropout=0.1,
    ):
        super().__init__()

        self.layers = nn.ModuleList(
            [EncoderBlock(features, n_heads, d_ff, dropout) for _ in range(n_layers)]
        )

        self.norm = LayerNormalization(features)

    def forward(self, x, src_mask):
        for layer in self.layers:
            x = layer(x, src_mask)

        return self.norm(x)

In [11]:
# Decoder Block
class DecoderBlock(nn.Module):
    def __init__(
        self,
        features,
        n_heads,
        d_ff,
        dropout=0.1,
    ):
        super().__init__()

        self.attention = MultiHeadAttention(features, n_heads, dropout)
        self.ffn = FFN(features, d_ff, dropout)
        self.residual = nn.ModuleList(
            [ResidualLayer(features, dropout) for _ in range(3)]
        )

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual[0](x, lambda x: self.attention(x, tgt_mask))
        x = self.residual[1](x, lambda x: self.attention(x, src_mask))

        x = self.residual[2](x, self.ffn)

        return x

In [12]:
class Decoder(nn.Module):
    def __init__(self, features, n_heads, d_ff, n_layers=8, dropout=0.1):
        super().__init__()

        self.layers = nn.ModuleList(
            [DecoderBlock(features, n_heads, d_ff, dropout) for _ in range(n_layers)]
        )

        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)

        return self.norm(x)

In [13]:
class Projection(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()

        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return self.fc(x)

In [14]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        src_embedding_size,
        tgt_embedding_size,
        d_model,
        n_heads,
        d_ff,
        n_layers,
        dropout=0.1,
    ):
        super().__init__()

        self.src_embedding = InputEmbedding(src_vocab_size, src_embedding_size)
        self.tgt_embedding = InputEmbedding(tgt_vocab_size, tgt_embedding_size)

        self.encoder = Encoder(d_model, n_heads, d_ff, n_layers, dropout)
        self.decoder = Decoder(d_model, n_heads, d_ff, n_layers, dropout)

        self.projection = Projection(tgt_vocab_size, d_model)

        # Weight tie
        self.projection.fc.weight = self.tgt_embedding.embedding.weight

    def encode(self, encoder_input, src_mask):
        return self.encoder(self.src_embedding(encoder_input), src_mask)

    def decode(self, decoder_input, encoder_output, src_mask, tgt_mask):
        return self.decoder(
            self.tgt_embedding(decoder_input), encoder_output, src_mask, tgt_mask
        )

    def forward(self, encoder_input, decoder_input, src_mask, tgt_mask):
        encoder_output = self.encode(encoder_input, src_mask)
        decoder_output = self.decode(decoder_input, encoder_output, src_mask, tgt_mask)

        return self.projection(decoder_output)

In [15]:
def build_transformer(
    src_vocab_size,
    tgt_vocab_size,
    src_embedding_size,
    tgt_embedding_size,
    d_model,
    n_heads,
    d_ff,
    n_layers,
    dropout=0.1,
):
    transformer = Transformer(
        src_vocab_size,
        tgt_vocab_size,
        src_embedding_size,
        tgt_embedding_size,
        d_model,
        n_heads,
        d_ff,
        n_layers,
        dropout,
    )

    # Initialize the weights
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return transformer

In [16]:
class Config:
    src_vocab_size = 10000
    tgt_vocab_size = 10000
    src_embedding_size = 512
    tgt_embedding_size = 512
    d_model = 512
    n_heads = 8
    d_ff = 2048
    n_layers = 6
    dropout = 0.1
    seq_len = 128

In [17]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

In [18]:
causal_mask(10)

tensor([[[ True, False, False, False, False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False, False, False],
         [ True,  True,  True,  True, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]]])

In [19]:
# test
config = Config()
transformer = build_transformer(
    config.src_vocab_size,
    config.tgt_vocab_size,
    config.src_embedding_size,
    config.tgt_embedding_size,
    config.d_model,
    config.n_heads,
    config.d_ff,
    config.n_layers,
    config.dropout,
)

x = torch.randint(0, 10000, (32, 128))
y = torch.randint(0, 10000, (32, 128))


src_mask = (x != 0).unsqueeze(1).unsqueeze(1)
tgt_mask = causal_mask(128)

assert transformer.encode(x, src_mask).shape == (32, 128, config.d_model)
assert transformer.decode(
    y, transformer.encode(x, src_mask), src_mask, tgt_mask
).shape == (32, 128, config.d_model)
assert transformer(x, y, src_mask, tgt_mask).shape == (32, 128, config.tgt_vocab_size)
print("All tests passed!")